# Validator Relationship Contagion Analysis

Comprehensive framework for analyzing MEV vulnerability contagion through validator relationships.

## Key Analysis Components:
1. **Leader Slot Concentration as Attractor** - Identify validator hotspots
2. **Specialized Exploitation through Validator-AMM Relationships** - Bot targeting patterns
3. **Exploitation of Slot Boundary Delays** - Cross-slot attack patterns (2Fast bots)
4. **Systematic Bot Targeting Ecosystem** - Attacker specialization and infrastructure advantages
5. **Contagion Pathways** - How attacks spillover across protocols
6. **Mitigation Effectiveness** - TWAP, commit-reveal, slot-level filtering recommendations

## Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict, Counter
from typing import Dict, List, Tuple, Set, Any
from dataclasses import dataclass, asdict
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✓ All imports successful")

## Data Models

In [ ]:
@dataclass
class ValidatorHotspot:
    """Represents a validator concentration hotspot."""
    validator_address: str
    total_mev_count: int
    unique_attackers: int
    unique_protocols: int
    concentration_ratio: float  # % of all MEV in dataset
    avg_attacks_per_slot: float
    slots_active: int
    risk_level: str  # HIGH, MEDIUM, LOW


@dataclass
class ContagionPath:
    """Represents a contagion pathway between validator-protocol pairs."""
    source_validator: str
    source_protocol: str
    target_validators: List[str]
    target_protocols: List[str]
    shared_attackers: List[str]
    contagion_strength: float  # 0-1, based on shared execution
    temporal_correlation: float  # How attacks cluster in time
    spillover_evidence: int  # Number of cross-slot jumps observed


@dataclass
class BotSpecialization:
    """Represents a bot's specialization pattern."""
    bot_address: str
    preferred_validators: Dict[str, int]  # validator -> count
    preferred_protocols: Dict[str, int]  # protocol -> count
    attack_types: Dict[str, int]  # sandwich, front_run, etc
    infrastructure_score: float  # Based on timing precision
    success_rate: float
    avg_profit_per_attack: float

print("✓ Data models defined")

## ValidatorContagionAnalyzer Class

Main analyzer class with methods for each analysis component.

In [ ]:
class ValidatorContagionAnalyzer:
    """
    Comprehensive validator relationship and MEV contagion analyzer.
    
    Identifies how validator relationships create systemic vulnerabilities
    that propagate across protocols and time periods.
    """
    
    def __init__(self, data_dir: str = "02_mev_detection", 
                 parallel_workers: int = 4):
        """
        Initialize the contagion analyzer.
        
        Parameters:
        -----------
        data_dir : str
            Directory containing MEV detection results
        parallel_workers : int
            Number of parallel workers for analysis
        """
        self.data_dir = Path(data_dir)
        self.mev_data = None
        self.validator_graph = None
        self.contagion_paths = []
        self.bot_specializations = {}
        self.hotspots = {}
        self.cross_slot_patterns = []
        self.parallel_workers = parallel_workers
        
    def load_mev_data(self, filepath: str = None):
        """Load MEV detection results."""
        if filepath is None:
            filepath = str(self.data_dir / "per_pamm_all_mev_with_validator.csv")
        
        self.mev_data = pd.read_csv(filepath)
        print(f"✓ Loaded {len(self.mev_data):,} MEV records")
        print(f"  Validators: {self.mev_data['validator'].nunique()}")
        print(f"  Attackers: {self.mev_data['attacker_signer'].nunique()}")
        print(f"  Protocols: {self.mev_data['amm_trade'].nunique()}")
        
        return self

print("✓ Analyzer class defined")

## Part 1: Validator Hotspot Identification

In [ ]:
# Add this method to ValidatorContagionAnalyzer class

def identify_validator_hotspots(self, top_n: int = 20,
                                concentration_threshold: float = 0.01) -> Dict[str, ValidatorHotspot]:
    """
    Identify validator hotspots - validators with disproportionate MEV concentration.
    
    Addresses: "Leader Slot Concentration as an Attractor"
    
    Parameters:
    -----------
    top_n : int
        Number of top validators to analyze
    concentration_threshold : float
        Minimum % of total MEV for HIGH risk classification
    
    Returns:
    --------
    Dict[str, ValidatorHotspot]
        Information about validator concentration hotspots
    """
    if self.mev_data is None:
        raise ValueError("Load MEV data first with load_mev_data()")
    
    total_mev = len(self.mev_data)
    
    # Calculate validator statistics
    validator_stats = {}
    
    for validator in self.mev_data['validator'].unique():
        val_data = self.mev_data[self.mev_data['validator'] == validator]
        
        mev_count = len(val_data)
        concentration = mev_count / total_mev
        unique_attackers = val_data['attacker_signer'].nunique()
        unique_protocols = val_data['amm_trade'].nunique()
        
        # Estimate slots active (based on unique slot/time combinations if available)
        if 'slot' in val_data.columns:
            slots_active = val_data['slot'].nunique()
        else:
            slots_active = len(val_data) // max(1, unique_attackers)  # Estimate
        
        avg_attacks_per_slot = mev_count / max(1, slots_active)
        
        # Risk classification
        if concentration >= concentration_threshold:
            risk_level = "HIGH"
        elif concentration >= concentration_threshold * 0.5:
            risk_level = "MEDIUM"
        else:
            risk_level = "LOW"
        
        hotspot = ValidatorHotspot(
            validator_address=validator,
            total_mev_count=mev_count,
            unique_attackers=unique_attackers,
            unique_protocols=unique_protocols,
            concentration_ratio=concentration,
            avg_attacks_per_slot=avg_attacks_per_slot,
            slots_active=slots_active,
            risk_level=risk_level
        )
        
        validator_stats[validator] = hotspot
    
    # Sort by MEV count and return top N
    sorted_validators = sorted(
        validator_stats.items(),
        key=lambda x: x[1].total_mev_count,
        reverse=True
    )[:top_n]
    
    self.hotspots = dict(sorted_validators)
    
    # Summary statistics
    print(f"\n{'='*70}")
    print(f"VALIDATOR HOTSPOT ANALYSIS")
    print(f"{'='*70}")
    print(f"\nTop {len(sorted_validators)} Validators by MEV Concentration:\n")
    
    for i, (validator, hotspot) in enumerate(sorted_validators, 1):
        print(f"{i:2d}. {validator[:16]}...")
        print(f"    MEV Count: {hotspot.total_mev_count:,} ({hotspot.concentration_ratio*100:.2f}%)")
        print(f"    Attackers: {hotspot.unique_attackers} | Protocols: {hotspot.unique_protocols}")
        print(f"    Avg Attacks/Slot: {hotspot.avg_attacks_per_slot:.2f} | Risk: {hotspot.risk_level}")
        print()
    
    return self.hotspots

# Attach to class
ValidatorContagionAnalyzer.identify_validator_hotspots = identify_validator_hotspots
print("✓ Validator hotspot identification method added")

## Part 2: Validator-AMM Relationship Analysis

In [ ]:
def analyze_validator_amm_contagion(self, min_shared_attacks: int = 2) -> Dict[str, Any]:
    """
    Analyze contagion through validator-AMM relationships.
    
    Addresses: "Specialized Exploitation through Validator-AMM Relationships"
    """
    if self.mev_data is None:
        raise ValueError("Load MEV data first")
    
    results = {
        'validator_protocol_pairs': [],
        'high_risk_combinations': [],
        'contagion_pathways': [],
        'protocol_vulnerability_clusters': [],
        'attacker_specialization': {}
    }
    
    # 1. Build validator-protocol matrix
    validator_protocol_matrix = defaultdict(lambda: defaultdict(int))
    validator_protocol_attackers = defaultdict(lambda: defaultdict(set))
    
    for _, row in self.mev_data.iterrows():
        validator = row['validator']
        protocol = row['amm_trade']
        attacker = row['attacker_signer']
        
        validator_protocol_matrix[validator][protocol] += 1
        validator_protocol_attackers[validator][protocol].add(attacker)
    
    # 2. Identify high-risk validator-protocol combinations
    all_pairs = []
    for validator, protocols in validator_protocol_matrix.items():
        for protocol, count in protocols.items():
            unique_attackers = len(validator_protocol_attackers[validator][protocol])
            pair_risk = count * unique_attackers  # Risk score = volume * diversity
            
            all_pairs.append({
                'validator': validator,
                'protocol': protocol,
                'attack_count': count,
                'unique_attackers': unique_attackers,
                'risk_score': pair_risk,
                'attackers': list(validator_protocol_attackers[validator][protocol])
            })
    
    # Sort by risk score
    all_pairs.sort(key=lambda x: x['risk_score'], reverse=True)
    
    results['validator_protocol_pairs'] = all_pairs[:50]
    results['high_risk_combinations'] = [p for p in all_pairs if p['attack_count'] >= min_shared_attacks][:20]
    
    # 3. Detect contagion pathways
    contagion_pathways = self._detect_validator_level_contagion(validator_protocol_attackers)
    results['contagion_pathways'] = contagion_pathways
    
    # 4. Identify protocol vulnerability clusters
    protocol_clusters = self._identify_protocol_vulnerability_clusters(validator_protocol_attackers, all_pairs)
    results['protocol_vulnerability_clusters'] = protocol_clusters
    
    # Print summary
    print(f"\n{'='*70}")
    print(f"VALIDATOR-AMM CONTAGION ANALYSIS")
    print(f"{'='*70}")
    print(f"\nHigh-Risk Validator-Protocol Combinations:\n")
    
    for pair in results['high_risk_combinations'][:10]:
        print(f"  {pair['validator'][:16]}... + {pair['protocol']}")
        print(f"    Attacks: {pair['attack_count']} | Unique Bots: {pair['unique_attackers']} | Risk: {pair['risk_score']}")
        print()
    
    print(f"\nDetected {len(results['contagion_pathways'])} Contagion Pathways")
    print(f"Identified {len(results['protocol_vulnerability_clusters'])} Vulnerability Clusters\n")
    
    return results

def _detect_validator_level_contagion(self, validator_protocol_attackers: Dict) -> List[dict]:
    """Detect contagion pathways where same attackers exploit multiple protocols."""
    contagion_paths = []
    
    for validator, protocol_dict in validator_protocol_attackers.items():
        for source_protocol, attackers in protocol_dict.items():
            for target_protocol, target_attackers in protocol_dict.items():
                if source_protocol != target_protocol:
                    shared_attackers = attackers & target_attackers
                    
                    if len(shared_attackers) > 0:
                        strength = len(shared_attackers) / len(attackers)
                        
                        contagion_paths.append({
                            'validator': validator,
                            'source_protocol': source_protocol,
                            'target_protocol': target_protocol,
                            'shared_attackers': list(shared_attackers),
                            'num_shared': len(shared_attackers),
                            'contagion_strength': strength
                        })
    
    return sorted(contagion_paths, key=lambda x: x['num_shared'], reverse=True)

def _identify_protocol_vulnerability_clusters(self, validator_protocol_attackers: Dict, all_pairs: List[Dict]) -> List[Dict]:
    """Identify clusters of protocols vulnerable through same validators."""
    clusters = []
    
    protocol_vulnerability = {}
    for pair in all_pairs:
        protocol = pair['protocol']
        if protocol not in protocol_vulnerability:
            protocol_vulnerability[protocol] = {'count': 0, 'validators': []}
        protocol_vulnerability[protocol]['count'] += pair['attack_count']
        protocol_vulnerability[protocol]['validators'].append(pair['validator'])
    
    protocols = list(protocol_vulnerability.keys())
    for i, prot1 in enumerate(protocols):
        for prot2 in protocols[i+1:]:
            common_validators = set(protocol_vulnerability[prot1]['validators']) & \
                               set(protocol_vulnerability[prot2]['validators'])
            
            if len(common_validators) > 0:
                clusters.append({
                    'protocol_pair': [prot1, prot2],
                    'shared_validators': list(common_validators),
                    'num_shared': len(common_validators),
                    'combined_risk': protocol_vulnerability[prot1]['count'] + protocol_vulnerability[prot2]['count']
                })
    
    return sorted(clusters, key=lambda x: x['combined_risk'], reverse=True)

ValidatorContagionAnalyzer.analyze_validator_amm_contagion = analyze_validator_amm_contagion
ValidatorContagionAnalyzer._detect_validator_level_contagion = _detect_validator_level_contagion
ValidatorContagionAnalyzer._identify_protocol_vulnerability_clusters = _identify_protocol_vulnerability_clusters
print("✓ Validator-AMM contagion analysis methods added")

## Part 3: Cross-Slot Pattern Detection

In [ ]:
def detect_cross_slot_patterns(self, slot_duration_ms: int = 400,
                               time_column: str = 'ms_time') -> Dict[str, Any]:
    """
    Detect exploitation of slot boundary delays - the 2Fast Bot pattern.
    
    Identifies multi-slot attack patterns where same attacker exploits
    timing across slot boundaries to maximize MEV.
    """
    if self.mev_data is None:
        raise ValueError("Load MEV data first")
    
    if 'slot' not in self.mev_data.columns or time_column not in self.mev_data.columns:
        print("⚠ Warning: 'slot' or time column not available for cross-slot analysis")
        return {'status': 'unavailable', 'reason': 'Missing slot or time columns'}
    
    results = {
        'multi_slot_attackers': [],
        'cross_slot_sandwiches': [],
        'slot_boundary_exploits': [],
        'temporal_attack_clusters': []
    }
    
    # 1. Identify attackers with trades spanning multiple slots
    cross_slot_trades = []
    
    for attacker in self.mev_data['attacker_signer'].unique():
        attacker_data = self.mev_data[self.mev_data['attacker_signer'] == attacker].copy()
        
        if len(attacker_data) < 3:
            continue
        
        unique_slots = attacker_data['slot'].nunique()
        if unique_slots > 1:
            slot_list = sorted(attacker_data['slot'].unique())
            
            for i in range(len(slot_list) - 1):
                slot_gap = slot_list[i+1] - slot_list[i]
                if slot_gap <= 3:
                    cross_slot_trades.append({
                        'attacker': attacker,
                        'slot_sequence': [slot_list[i], slot_list[i+1]],
                        'slot_gap': slot_gap,
                        'attack_type': 'potential_2fast_bot'
                    })
    
    results['multi_slot_attackers'] = cross_slot_trades[:50]
    
    # 2. Identify cross-slot sandwiches
    if 'fat_sandwich' in self.mev_data.columns:
        fat_sandwich_data = self.mev_data[self.mev_data['fat_sandwich'] == True]
        
        for attacker in fat_sandwich_data['attacker_signer'].unique():
            attacker_fat = fat_sandwich_data[fat_sandwich_data['attacker_signer'] == attacker]
            slots = sorted(attacker_fat['slot'].unique())
            
            if len(slots) > 1:
                results['cross_slot_sandwiches'].append({
                    'attacker': attacker,
                    'slots_involved': slots,
                    'fat_sandwich_count': len(attacker_fat),
                    'protocols': list(attacker_fat['amm_trade'].unique())
                })
    
    # 3. Detect slot boundary timing exploits
    if time_column in self.mev_data.columns:
        boundary_exploits = []
        
        for slot in self.mev_data['slot'].unique():
            slot_data = self.mev_data[self.mev_data['slot'] == slot].copy()
            
            if len(slot_data) > 0:
                min_time = slot_data[time_column].min()
                slot_data['time_in_slot'] = slot_data[time_column] - min_time
                
                boundary_trades = slot_data[slot_data['time_in_slot'] > slot_duration_ms * 0.9]
                
                if len(boundary_trades) > 0:
                    boundary_exploits.append({
                        'slot': slot,
                        'boundary_trades': len(boundary_trades),
                        'attackers': boundary_trades['attacker_signer'].nunique(),
                        'protocols': boundary_trades['amm_trade'].nunique()
                    })
        
        results['slot_boundary_exploits'] = sorted(boundary_exploits, key=lambda x: x['boundary_trades'], reverse=True)[:20]
    
    print(f"\n{'='*70}")
    print(f"CROSS-SLOT PATTERN DETECTION (2Fast Bot Analysis)")
    print(f"{'='*70}")
    print(f"\nMulti-Slot Attacker Patterns: {len(results['multi_slot_attackers'])}")
    print(f"Cross-Slot Fat Sandwiches: {len(results['cross_slot_sandwiches'])}")
    print(f"Slot Boundary Exploits: {len(results['slot_boundary_exploits'])}\n")
    
    return results

ValidatorContagionAnalyzer.detect_cross_slot_patterns = detect_cross_slot_patterns
print("✓ Cross-slot pattern detection method added")

## Part 4: Bot Ecosystem Mapping

In [ ]:
def map_bot_ecosystem(self, top_n_bots: int = 50) -> Dict[str, Any]:
    """
    Map the systematic bot ecosystem.
    
    Identifies bot specialization, infrastructure advantages, and
    competitive positioning in the Solana MEV landscape.
    """
    if self.mev_data is None:
        raise ValueError("Load MEV data first")
    
    results = {
        'bot_count': 0,
        'top_bots': [],
        'bot_specialization_matrix': [],
        'validator_targeting': [],
        'infrastructure_indicators': {},
        'ecosystem_summary': {}
    }
    
    # 1. Basic bot statistics
    bot_stats = {}
    
    for bot in self.mev_data['attacker_signer'].unique():
        bot_data = self.mev_data[self.mev_data['attacker_signer'] == bot]
        
        attack_count = len(bot_data)
        unique_validators = bot_data['validator'].nunique()
        unique_protocols = bot_data['amm_trade'].nunique()
        
        # Attack type distribution
        attack_types = {}
        for col in ['sandwich', 'front_running', 'back_running', 'fat_sandwich']:
            if col in bot_data.columns:
                attack_types[col] = int(bot_data[col].sum())
        
        # Success rate
        if 'confidence' in bot_data.columns:
            try:
                confidence_numeric = pd.to_numeric(bot_data['confidence'], errors='coerce')
                success_rate = confidence_numeric.mean()
                if pd.isna(success_rate):
                    success_rate = 0.5
            except:
                success_rate = 0.5
        else:
            success_rate = 0.5
        
        # Profitability
        if 'net_profit_sol' in bot_data.columns:
            total_profit = bot_data['net_profit_sol'].sum()
            avg_profit = bot_data['net_profit_sol'].mean()
        else:
            total_profit = 0
            avg_profit = 0
        
        # Timing precision
        timing_precision = self._calculate_timing_precision(bot_data)
        
        bot_stats[bot] = {
            'attack_count': attack_count,
            'unique_validators': unique_validators,
            'unique_protocols': unique_protocols,
            'attack_types': attack_types,
            'success_rate': success_rate,
            'total_profit_sol': total_profit,
            'avg_profit_sol': avg_profit,
            'timing_precision': timing_precision,
            'infrastructure_score': self._calculate_infrastructure_score(timing_precision, success_rate, unique_validators),
            'preferred_validators': dict(bot_data['validator'].value_counts().head(5)),
            'preferred_protocols': dict(bot_data['amm_trade'].value_counts().head(5))
        }
    
    results['bot_count'] = len(bot_stats)
    
    # 2. Get top bots
    top_bots_list = sorted(bot_stats.items(), key=lambda x: x[1]['attack_count'], reverse=True)[:top_n_bots]
    
    results['top_bots'] = [
        {
            'bot': bot,
            'attack_count': stats['attack_count'],
            'validators': stats['unique_validators'],
            'protocols': stats['unique_protocols'],
            'success_rate': stats['success_rate'],
            'infrastructure_score': stats['infrastructure_score'],
            'total_profit_sol': stats['total_profit_sol']
        }
        for bot, stats in top_bots_list
    ]
    
    # 3. Infrastructure metrics
    precision_scores = [stats['timing_precision'] for stats in bot_stats.values()]
    infrastructure_scores = [stats['infrastructure_score'] for stats in bot_stats.values()]
    
    results['infrastructure_indicators'] = {
        'mean_timing_precision_ms': np.mean(precision_scores),
        'mean_infrastructure_score': np.mean(infrastructure_scores),
        'high_quality_bots': sum(1 for s in infrastructure_scores if s > 0.7)
    }
    
    # Print summary
    print(f"\n{'='*70}")
    print(f"BOT ECOSYSTEM MAPPING")
    print(f"{'='*70}")
    print(f"\nTotal Unique Bots: {results['bot_count']}")
    print(f"High-Infrastructure Bots (score > 0.7): {results['infrastructure_indicators']['high_quality_bots']}")
    print(f"Mean Timing Precision: {results['infrastructure_indicators']['mean_timing_precision_ms']:.2f}ms")
    print(f"\nTop 10 Bots by Activity:\n")
    
    for i, bot_info in enumerate(results['top_bots'][:10], 1):
        print(f"{i:2d}. {bot_info['bot'][:16]}...")
        print(f"    Attacks: {bot_info['attack_count']} | Validators: {bot_info['validators']} | Protocols: {bot_info['protocols']}")
        print(f"    Success Rate: {bot_info['success_rate']:.2%} | Infrastructure: {bot_info['infrastructure_score']:.2f}")
        print()
    
    return results

def _calculate_timing_precision(self, bot_data: pd.DataFrame) -> float:
    """Calculate timing precision as indicator of infrastructure quality."""
    if 'time_diff_ms' in bot_data.columns:
        timing_data = bot_data['time_diff_ms'].dropna()
        if len(timing_data) > 0:
            precision = 1 / (1 + timing_data.std() / 10)
            return min(precision * 100, 100)
    return 50

def _calculate_infrastructure_score(self, timing_precision: float, success_rate: float, validator_diversity: int) -> float:
    """Calculate bot infrastructure quality score (0-10 scale)."""
    timing_component = (timing_precision / 100) * 3
    success_component = success_rate * 4
    diversity_component = min(validator_diversity / 50, 1) * 3
    return timing_component + success_component + diversity_component

ValidatorContagionAnalyzer.map_bot_ecosystem = map_bot_ecosystem
ValidatorContagionAnalyzer._calculate_timing_precision = _calculate_timing_precision
ValidatorContagionAnalyzer._calculate_infrastructure_score = _calculate_infrastructure_score
print("✓ Bot ecosystem mapping methods added")

## Part 5: Mitigation Recommendations

In [ ]:
def generate_mitigation_recommendations(self) -> Dict[str, Any]:
    """
    Generate mitigation recommendations based on the analysis.
    
    Implements recommendations for:
    - Slot-level MEV filtering
    - TWAP (Time-Weighted Average Price) implementation
    - Commit-reveal schemes
    - Validator diversity requirements
    """
    if self.mev_data is None:
        raise ValueError("Load MEV data first")
    
    recommendations = {
        'slot_level_filtering': self._recommend_slot_filtering(),
        'twap_implementation': self._recommend_twap(),
        'commit_reveal_scheme': self._recommend_commit_reveal(),
        'validator_diversity': self._recommend_validator_diversity(),
        'bot_detection_rules': self._generate_bot_detection_rules(),
        'implementation_priority': [
            {
                'rank': 1,
                'strategy': 'Slot-Level MEV Filtering',
                'impact': 'HIGH',
                'effort': 'MEDIUM',
                'estimated_reduction': '60-70% of coordinated attacks'
            },
            {
                'rank': 2,
                'strategy': 'TWAP-Based Oracle Updates',
                'impact': 'HIGH',
                'effort': 'MEDIUM',
                'estimated_reduction': '50-60% of oracle-timed attacks'
            },
            {
                'rank': 3,
                'strategy': 'Commit-Reveal Transactions',
                'impact': 'MEDIUM',
                'effort': 'HIGH',
                'estimated_reduction': '80-90% of sandwich attacks'
            },
            {
                'rank': 4,
                'strategy': 'Validator Diversity Enforcement',
                'impact': 'MEDIUM',
                'effort': 'LOW',
                'estimated_reduction': '20-30% of concentrated attacks'
            }
        ]
    }
    
    print(f"\n{'='*70}")
    print(f"MITIGATION RECOMMENDATIONS")
    print(f"{'='*70}\n")
    
    for item in recommendations['implementation_priority']:
        print(f"{item['rank']}. {item['strategy']}")
        print(f"   Impact: {item['impact']} | Effort: {item['effort']}")
        print(f"   Estimated Reduction: {item['estimated_reduction']}")
        print()
    
    return recommendations

def _recommend_slot_filtering(self) -> Dict[str, Any]:
    return {
        'strategy': 'Implement slot-level MEV filtering at validator level',
        'targets': ['High-concentration validators', 'BisonFi and HumidiFi protocols'],
        'complexity': 'MEDIUM'
    }

def _recommend_twap(self) -> Dict[str, Any]:
    return {
        'strategy': 'Implement Time-Weighted Average Price (TWAP) oracle updates',
        'complexity': 'MEDIUM',
        'expected_impact': '50-60% reduction in back-run attacks'
    }

def _recommend_commit_reveal(self) -> Dict[str, Any]:
    return {
        'strategy': 'Implement commit-reveal transaction scheme',
        'complexity': 'HIGH',
        'user_experience_impact': 'Requires additional transaction + increased latency'
    }

def _recommend_validator_diversity(self) -> Dict[str, Any]:
    return {
        'strategy': 'Enforce validator diversity for critical operations',
        'complexity': 'LOW',
        'expected_impact': '20-30% reduction in concentrated attacks'
    }

def _generate_bot_detection_rules(self) -> Dict[str, Any]:
    return {
        'high_infrastructure_bot_detection': {
            'indicators': ['Timing precision < 5ms', 'Success rate > 80%', 'Infrastructure score > 7.0'],
            'action': 'Flag for validator MEV filtering'
        },
        'specialized_attack_pattern': {
            'indicators': ['Attacks concentrated on 1-2 protocols (>70%)', 'Concentrated on 1-3 validators'],
            'action': 'Recommend protocol-level defenses'
        }
    }

ValidatorContagionAnalyzer.generate_mitigation_recommendations = generate_mitigation_recommendations
ValidatorContagionAnalyzer._recommend_slot_filtering = _recommend_slot_filtering
ValidatorContagionAnalyzer._recommend_twap = _recommend_twap
ValidatorContagionAnalyzer._recommend_commit_reveal = _recommend_commit_reveal
ValidatorContagionAnalyzer._recommend_validator_diversity = _recommend_validator_diversity
ValidatorContagionAnalyzer._generate_bot_detection_rules = _generate_bot_detection_rules
print("✓ Mitigation recommendation methods added")

def export_contagion_graph(self, output_file: str = 'validator_contagion_graph.json'):
    """Export validator contagion relationships as network graph."""
    if not self.hotspots:
        print("⚠ No hotspots analyzed. Run identify_validator_hotspots() first.")
        return
    
    graph_data = {
        'nodes': [],
        'edges': [],
        'metadata': {
            'total_validators_analyzed': len(self.hotspots),
            'total_records': len(self.mev_data) if self.mev_data is not None else 0
        }
    }
    
    for validator, hotspot in self.hotspots.items():
        graph_data['nodes'].append({
            'id': validator,
            'type': 'validator',
            'mev_count': hotspot.total_mev_count,
            'concentration': hotspot.concentration_ratio,
            'risk_level': hotspot.risk_level
        })
    
    with open(output_file, 'w') as f:
        json.dump(graph_data, f, indent=2)
    
    print(f"✓ Exported contagion graph to {output_file}")
    return graph_data

def generate_summary_report(self) -> Dict[str, Any]:
    """Generate comprehensive summary report of contagion analysis."""
    report = {
        'analysis_timestamp': pd.Timestamp.now().isoformat(),
        'data_records': len(self.mev_data) if self.mev_data is not None else 0,
        'validators_affected': self.mev_data['validator'].nunique() if self.mev_data is not None else 0,
        'unique_attackers': self.mev_data['attacker_signer'].nunique() if self.mev_data is not None else 0,
        'hotspots_identified': len(self.hotspots),
        'key_findings': [],
        'status': 'Analysis complete'
    }
    
    if self.hotspots:
        top_hotspot = list(self.hotspots.values())[0]
        report['key_findings'].append(
            f"Top validator {list(self.hotspots.keys())[0][:16]}... "
            f"accounts for {top_hotspot.concentration_ratio*100:.1f}% of all MEV activity"
        )
    
    return report

ValidatorContagionAnalyzer.export_contagion_graph = export_contagion_graph
ValidatorContagionAnalyzer.generate_summary_report = generate_summary_report
print("✓ Utility methods added")

# Initialize analyzer
analyzer = ValidatorContagionAnalyzer(
    data_dir="../02_mev_detection"
)

print("✓ Analyzer initialized")

# Load data
try:
    analyzer.load_mev_data()
except FileNotFoundError:
    print("⚠ Could not find MEV detection data. Please check the data path.")

### Export and Summarize Results

In [ ]:
# Export results
if analyzer.mev_data is not None:
    analyzer.export_contagion_graph('validator_contagion_graph.json')
    summary = analyzer.generate_summary_report()
    
    print("\n" + "="*70)
    print("ANALYSIS COMPLETE")
    print("="*70)
    print(f"\nSummary Report:")
    print(f"  Timestamp: {summary['analysis_timestamp']}")
    print(f"  Total Records: {summary['data_records']:,}")
    print(f"  Validators Affected: {summary['validators_affected']}")
    print(f"  Unique Attackers: {summary['unique_attackers']}")
    print(f"  Hotspots Identified: {summary['hotspots_identified']}")